In [0]:
%run "./weather_ETL_schemas"

In [0]:
from pyspark.sql.functions import col, from_unixtime, lit
from datetime import datetime
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, TimestampType, ArrayType

In [0]:
# Using decorator for implementing the log table
def update_log(func):
    def wrapper_function(*args):
        
        # Extracting the necessary fields
        id = str(uuid.uuid4())
        load_type = args[0]
        table_name = args[1]
        comments = args[2]
        process_start_time = datetime.now()
        status = 'EXTRACTING'
        
        load_run_id = id
        created_by = 'Ananda Thakur'
        created_on = process_start_time
        
        # Creating log record for start of process
        log_data = [(id, load_type, table_name, process_start_time, None, status, comments, None, None, created_on, created_by)]
        log_df = spark.createDataFrame(log_data, schema=get_log_schema())
        log_df.write.format('delta').mode('append').saveAsTable('log_table_new')
        
        process_start_time = datetime.now()
        created_on = process_start_time
        
        # Creating necessary columns and saving to table and returning ERROR if the process fails
        try:
            df, start_dt, end_dt = func(*args[3:])

            df = df.withColumn('load_run_id', lit(load_run_id))
            df = df.withColumn('created_on', lit(created_on))
            df = df.withColumn('created_by', lit(created_by))

            df.write.format('delta').mode('append').saveAsTable(table_name)

            status = 'COMPLETED'
        except:
            status = 'ERROR'
        
        process_end_time = datetime.now()
        
        # Creating log record for start of process
        log_data = [(id, load_type, table_name, process_start_time, process_end_time, status, comments, start_dt, end_dt, created_on, created_by)]
        log_df = spark.createDataFrame(log_data, schema=get_log_schema())
        log_df.write.format('delta').mode('append').saveAsTable('log_table_new')
        
        return df
    return wrapper_function